## SquamataAssemballyAMT - Jupyter notebook for batch releasing Audio Magnetotellic (AMT) data to ScienceBase

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load MT EDI file.
- Clean up and reformat harvested values to be XML metadata complient.
- Create User Editable Keywords Listing
- Create entity and attribute XML markup.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Fix procstep section; do we need this function to collect file information or do we plan on handeling this with boilerplate.

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAMT.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.1 
- created stack loop for batch mode processing
- harvest process dates
- seperatate out different text fils for A+E, cross power, fourier, etc.
- correct boiler plate for A+E and process steps

v1.2
- fix procedure date harvest bug, typo in first print statement

v1.3
- fix procedure date harvest to harvest file modified date instaed of file created date using "m" time instaed of "c" time; e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))
- Fix attribute description boiler plate for BP and SD text files.
- Add entity and attributes for edi file
- add entity and attributes for BP and SD files
- correct loops to create entity and attributes for all FC and TC files, not just the last one in the loop.

v1.4
 - fix attribute description boiler plate for BP.txt and SD.txt (removed the word 'binary')
 - fix attribute description boiler plate for RSP files (BF's units are Volts/Gamma, EF's units are Volts)
 - fix attribute description boiler plate for FC files (replaced word 'Time Series' with 'Fourier Coefficient')
 - fix attribute description boiler plate for BP files (replaced word 'Time Series' with 'Cross Power')
 
v1.5
 - assign the edi file name to variable 'EdiFileName' so it is correctly harvested by the XML template {EdiFileName} tag. 
 - Add code to harvest TS1 file modified time,  {CollectionDate} for "Acquire multiple high frequency time series files TS1 file extension)..." process step. 

v1.6
 - fix xml tagging errors in the procstep section of the metadata
 - fix indexing so that it pulls in all 31 values for the entity and attribute calculations of the EDI file
 
v1.7
 - fix xml definition and unit errors in the Make Array attributes for impedances
 - fix xml definition and unit errors in the Make Array attributes for tipper parameters
 
v1.8
 - correct computed parameter definition for both the impedance VAR and the tipper VAR attributes so that search on "R" in file name does not pick up .VAR files.
 - cleaned up formatting issues so that the correct number of tabs appear in the final xml output between tags
 
v1.9
 - Correct Tipper varience attribute output for var.exp.
 - Correct pandas data frame problem regarding min/max values for computed MT parameters
 
v1.10
 - revise MT-MetadataSilvertonTemplate-v1-10 to reconcile MRJ's review of Silverton AMT data
 - correct Create editable keywords to reconcile MRJ's review of Silverton AMT data 
 - correct missing Tipper TXR.EXP and TYR.EXP attributes
 - correct assign variables for xml file find and replace to include range of each COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 - correct Create the Entity and Attributes for the .txt files from detailed to overview
 - correct Create the Entity and Attributes for the FC listing from detailed to overview
 - correct Create the Entity and Attributes for the Time series .TS? listing from detailed to overview
 - correct Create the Entity and Attributes for the .BP? listing from detailed to overview
 - correct SD files Entity and Attributes from detailed to overview
 - add variables for Replace values of current metadata template with the appropriate values  to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 - need to fix Import entity and attributes - !****COMPUTED PARAMETERS****! to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 
v1.11
 - fixed Import entity and attributes - !****COMPUTED PARAMETERS****! to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters.
 
v1.12
 - revise and sort themekeys to obey Stu Giles review of Arkansas AMT metadate review.
 
v1.13
 - Brian fixed errors with entity and attributes.
 
v1.14
 - corrected spelling of "sychronous" to "synchronous" in Entity and Attribute Overview
 - way to add URLs for overview section??? - just added a variable for now
 - correct the number of sig figs for bounding coordinates.  Truncate or round.  Check In [14] versus In [17]
 - corrected Vertical Positional Accuracy Report: "Elevations were determined from USGS, The National Map, Bulk Point Query Service based" to "Elevations were determined from The USGS National Map Bulk Point Query Service based"
 - corrected min max value reversal for 'FREQ' in the template
 - corrected wrong value being read in for Phx Max being read in as "Low F'

v1.15
 - fixed None Thesaurus keywords as per Stu Giles review of AMT Arizona data release 
 - fixed Place Keyword (Arkansas --> Arizona)

v1.16
 - fixed Place Keyword (Arizona --> Colorado)
 - fixed other project related Place Keywords
 
 v1.17
 - Check SD* files for data and catch cases with aborted runs.
 - Create udom markup tag boiler plate for E&A text for aborted run files.
 
 v1.18
 - fix SD* nested looping problem of loopoing through the list the number of list times by accident causing way too many E and A listings per file.
  
 v1.19
 - modify themes for MT data only.
 
 v1.20
 - correct prtoblems with parameter lists different lenghts than files before.

 


In [1]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [1]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

C:\ProgramData\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# 1) Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [2]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
mtDataPath = r"C:\CurrentWork\DataReleases\Animas\MT2019 data release\Animas-MT2019" #The 'r' signifies a string literal. Use for paths.
mtMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\Animas"
mtMetaDataTemplateName = "MT-MetadataAnimasTemplatev1-19.xml"

In [3]:
#Check Paths for the fun of it
print ('The MT Data Path is: ' + '"' + mtDataPath + '"')
mtMetaDataTemplatePath + "\\" + mtMetaDataTemplateName

The MT Data Path is: "C:\CurrentWork\DataReleases\Animas\MT2019 data release\Animas-MT2019"


'C:\\CurrentWork\\DataReleases\\Animas\\MT-MetadataAnimasTemplatev1-19.xml'

## Now, let's explore our data. 
- What files do we have? 
- What files do we import values from?

In [4]:
#Explore data files and directory structure hosted below the provided provided parent data directory

In [5]:
#Produce directory listing of station (SB Object Children)
#Either set up the root directory with station subdirectories only or delete non-station directories from the list array
#mtDataDirList = os.walk(mtDataPath)
#mtDataDirList = [entry.path for entry in os.scandir(mtDataPath) if entry.is_dir()]
mtDataDirList = next(os.walk(mtDataPath))[1]
mtDataDirList

['mtag01',
 'mtag02',
 'mtag03',
 'mtag04',
 'mtag05',
 'mtag06',
 'mteg06',
 'mteg07',
 'mteg08',
 'mtkm01',
 'mtkm02',
 'mtkm03',
 'mtkm04',
 'mtkm05',
 'mtmb06',
 'mtmb07',
 'mtmb08',
 'mtmb09',
 'mtmc01',
 'mtsf01',
 'mtsf03',
 'mtsf04',
 'mtsf05',
 'mtsf06']

# We can select an indivdual station to test or run code for all stations in batch mode.  

In [106]:
#!!!!!!!!!Run this code and select A single station !!!!!!!!!
#!!!!!!!!!Skip the loop cell below and run cells sequentually !!!!!!!!!!
#!!!!!!!!!Use Shift-Enter to then execute each cell one by one!!!!!!!!!! 
mtStationPath = mtDataPath + '\\' + mtDataDirList[0]
mtStationPath

'C:\\CurrentWork\\DataReleases\\Animas\\MT2019 data release\\Animas-MT2019\\mtag01'

In [107]:
#Insert the child URL Here
childURL = 'https://www.sciencebase.gov/catalog/item/5efa5e5782ced62aaaee1e9b'

# ----------------- NEXT STEP IS BROKEN LOOP (SKIP!!!) ----------------------------------

In [10]:
#!!!!!!!!!!!!!!!!!!  Run this code to loop through ALL Data Sets !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!This code will produce metadata files for ALL stations in batch mode!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!  Skip running this code if processing a single station !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#for i in range (0,len(mtDataDirList)):
#    mtStationPath = mtDataPath + '\\' + mtDataDirList[i]
#    from IPython.display import Javascript
#    display(Javascript('IPython.notebook.execute_cells_below()'))

# -----------------------------------------------------------------------------------------------------------

In [108]:

#Look for EDI file to load
ediList = glob.glob(os.path.join(mtStationPath, '**/*MT*.edi'),  recursive=True)
ediPath = ediList[0]
#ediList
print ('EDI File Path:\n' + ediPath)
ediPathArray = ediPath.split('\\')
ediFile = str(ediPathArray[len(ediPathArray)-1])
print ('EDI File:\n' + ediFile)
EdiFileName = ediFile
#ediPathArray

EDI File Path:
C:\CurrentWork\DataReleases\Animas\MT2019 data release\Animas-MT2019\mtag01\USA-Colorado-Silverton_Caldera-2019-MTAG01.edi
EDI File:
USA-Colorado-Silverton_Caldera-2019-MTAG01.edi


## Harvest from the MT EDI file. 
### Parameters include:
- ProductId=USA-New_Mexico-Rio_Grande_Rift-San_Luis_Basin-2009-AMT01.edi
- ExternalUrl Url=https://doi.org/10.5066/F72F7MQ7
- Attachment Filename=https://pubs.usgs.gov/of/2011/1264/report/OF11-1264.pdf
- Survey Purpose Description: 
- Data Description:
- Citation Title=Audiomagnetotelluric data, Taos Plateau Volcanic Field, New Mexico
- Citation Authors=Chad E. Ailes, Brian D. Rodriguez
- Citation Year=2011
- YearCollected=2009
- Country=USA                                  
- Ellipsoid=Clarke 1866                                                          
- Location datum=NAD27 CONUS                                                     
- SITE LATITUDE=36.752985000                                                     
- SITE LONGITUDE=-105.560966167                                                  
- Elevation units="meters"=2608.00                                                                     
- Start=2009-07-21T19:52:03 UTC/GMT
- End=2009-07-21T20:34:20 UTC/GMT
- ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 
- Entities and Attributes:
    - FREQUENCIES
    - IMPEDANCE ROTATION ANGLES
    - IMPEDANCES
    - TIPPER PARAMETERS
    - COMPUTED PARAMETERS


## Lets now import and index values from the EDI Files
- We need these values for the metadata template.  
- We also want to run stats on some of these values for the entity and attributes section

In [109]:
    #Load EDI File and Read It
    ediFile = open(ediPath, 'r')
    ediContent = ediFile.read()
    ediFile.close()
    print(ediContent)


>HEAD                                                                           
                                                                                
  DATAID="Arrastra Gulch"                                                       
  ACQBY=USGS                                                                    
  ACQDATE=2019-07-17
  STATE=Colorado                                                                
  COUNTY="San Juan"                                                             
  UNITS=M                                                                       
  STDVERS=1.0                                                                   
  PROGVERS=GEOTOOLS_2.3                                                         
  PROGDATE=09/16/94                                                             
                                                                                
>INFO   MAXLINES=1000                                                           
       

In [110]:
#Now assign values to the SB MetaDataWizard Template unknowns
list_ = ediContent.splitlines()
list_length = len (list_)

# there are probally easier ways to loop through the below but I like having it all hard coded upfront
# it's easire to track an change for me
# Use the examples provided below to extract additional parameters
# Not that all variables being collected are not necessarily used in populating the template.
# Note that values can be hardcoded into the metadata xml template and/or harvested from the edi file

for X in list_:
  if "ProductId" in X:
    productArray = X.split('=')
    productIdArray = productArray[1].split('.')
    productId = productIdArray[0]
    # We may want to reformat this are parse out this name further for use with a root name based on the Data Release Title?
    productId = productId.replace("-", " ")
    productId = productId.replace("_", " ")
    drTitle = productId
    print ('Child Title: ' + productId)
  elif "ExternalUrl Url" in X:
    externalURLArray = X.split('=')
    externalURL = externalURLArray[1]
    print ('<onlink>: ' + externalURL)
  elif "STATE" in X:
    stateArray = X.split('=')
    state = stateArray[1].replace('"', "") #remove quotes around state
    print ('State: ' + state)
  elif "COUNTY" in X:
    countyArray = X.split('=')
    county = countyArray[1]
    print ('County: ' + county)
  elif "Start" in X:
    startArray = X.split('=')
    start = startArray[1]
    print ('Start: ' + start)
  elif "End" in X:
    endArray = X.split('=')
    end = endArray[1]
    print ('End: ' + end)
  elif "Attachment Filename" in X and "http" in X:
    lgwrklinkArray = X.split('=')
    lgwrklink = lgwrklinkArray[1]
    print ('Attachment Filename Link: ' + lgwrklink)
  elif "Citation Title" in X:
    citTitArray = X.split('=')
    citTit = citTitArray[1]
    print ('Citation Title: ' + citTit)
  elif "Citation Authors" in X:
    citNamesArray = X.split('=')
    citAuthorsArray = citNamesArray[1].split(',')
    for author in citAuthorsArray:
     author = author.strip()
     print ('Author: '+ author)
  elif "Citation Year" in X:
    citYearArray = X.split('=')
    citYear = citYearArray[1]
    print ('Citation Year: ' + citYear)
  elif "YearCollected" in X:
    yearColArray = X.split('=')
    yearCol = yearColArray[1]
    print ('Year Collected: ' + yearCol)
  elif "Ellipsoid" in X:
    ellipsoidArray = X.split('=')
    ellipsoid = ellipsoidArray[1]
    print ('Ellipsoid: ' + ellipsoid)
  elif "Location datum" in X:
    locDatumArray = X.split('=')
    locDatum = locDatumArray[1]
    print ('Local datum: ' + locDatum)
  elif "SITE LATITUDE" in X:
    sitLatArray = X.split('=')
    sitLat = sitLatArray[1] #may need to trim extra spaces !!!
    sitLat = float(sitLat) #round to correct number of sigfigs - do this above; comment this in if you want to use this function 
    sitLat = round(sitLat, 6)
    print ('Site latitude: ' + str(sitLat))
  elif "SITE LONGITUDE" in X:
    sitLonArray = X.split('=')
    sitLon = sitLonArray[1] # may need to trim extra spaces !!!
    sitLon = float(sitLon) #round to correct number of sigfigs - do this above; comment this in if you want to use this function 
    sitLon = round(sitLon, 6)
    print ('Site longitude: ' + str(sitLon))
  elif "Elevation units" in X:
    elevationStringArray = X.split('=')
    siteElevation = elevationStringArray[2] 
    print ('Site Elevation: ' + siteElevation)
    elevationUnits = elevationStringArray[1].replace('"', "")
    print ('Elevation Units: ' + elevationUnits)
# Code below returns values that occupy more than one line
    
for i in range(list_length):
 value = list_[i] 
 if value.replace(" ", "") == 'SurveyPurposeDescription:':
  startIndPurpose = i + 1
   #print ('startIndPurpose: ' + str(startIndPurpose))
 if value.replace(" ", "") == 'DataDescription:':
   endIndPurpose = i - 1
   #print ('endIndPurpose: ' + str(endIndPurpose))
purpose = list_[startIndPurpose]
for j in range(startIndPurpose + 1,endIndPurpose): 
    purpose = purpose + list_[j]
    purposeClean = re.sub(' +', ' ',purpose)
print ('\nAbstract:\n\t' + purposeClean)

for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == 'DataDescription:':
   startIndDescription = k + 1
   #print ('startIndDescription: ' + str(startIndDescription))
 if value.replace(" ", "") == 'FILECREATOR:':
   endIndDescription = k - 9
   #print ('endIndDescription: ' + str(endIndDescription))
description = list_[startIndDescription]
for l in range(startIndDescription + 1,endIndDescription): 
    description = description + list_[l]
    descriptionClean = re.sub(' +', ' ',description)
print ('\nPurpose:\n\t' + descriptionClean) 


State: Colorado                                                                
County: "San Juan"                                                             
Child Title: USA Colorado Silverton Caldera 2019 MTAG01
<onlink>: https://doi.org/10.5066/P92HEP1K 
Citation Title: Magnetotelluric sounding data in the Silverton Caldera complex, Colorado, 2019
Author: James Reitman
Author: Brian D. Rodriguez
Author: and Philip J. Brown
Author: II.
Citation Year: 2020                                                             
Year Collected: 2019
Ellipsoid: WGS84                                                                
Local datum: WGS84                                                           
Site latitude: 37.82606
Site longitude: -107.63718
Site Elevation: 2933.02                                               
Elevation Units: meters
Start: 2019-07-17T22:27:58 UTC/GMT
End: 2019-07-17T23:33:38 UTC/GMT  
Start: 2019-07-17T22:27:58 UTC/GMT
End: 2019-07-17T23:33:38 UTC/GMT  

Abstract

In [111]:
#Reformat product ID to more correctly reflect child (station) name
arrChildID = productId.split(' ')
strChildID = arrChildID[-1]
strChildID


'MTAG01'

In [112]:
# Now let's format the start time and end time to be what the XML file wants for <begdate> and <enddate>

begdateArr = start.split(' ')
begdate_str = begdateArr[0]
begdate_obj = dateutil.parser.parse(begdate_str)
begdate = begdate_obj.strftime('%Y%m%d')
print('<begdate> ', begdate) 

enddateArr = end.split(' ')
enddate_str = enddateArr[0]
enddate_obj = dateutil.parser.parse(enddate_str)
enddate = enddate_obj.strftime('%Y%m%d')
print('<enddate> ', enddate) 


<begdate>  20190717
<enddate>  20190717


In [113]:
#Now we reformat the lat  and longitude to 6 sig figs as well as trim of any extra spaces
#Brian seems to be stripping out the sig figs now so there is no need.  
#So this strip doesn't seem to work - let's try it below using the round-up function.
sitLat = str(sitLat)
sitLat = sitLat.strip()
sitLon = str(sitLon)
sitLon = sitLon.strip()

print ('Site latitude: ' + sitLat)
print ('Site longitude: ' + sitLon)

Site latitude: 37.82606
Site longitude: -107.63718


In [114]:
# Now Reformat county by trimming the extra spaces
county = county.strip()
# comment out below.  This is because the edi file now has the county in quotes and I don't want to strip those out
county = "San Juan"
print ('County: ' + county)

County: San Juan


In [115]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = widgets.Textarea(
    value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>\n\t\t\t\t<themekey>geoscientificInformation</themekey>' \
    + '\n\t\t\t\t<themekey>economy</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>' \
    + '\n\t\t\t\t<themekey>Department of the Interior</themekey>' \
    + '\n\t\t\t\t<themekey>DOI</themekey>' \
    + '\n\t\t\t\t<themekey>U.S. Geological Survey</themekey>' \
    + '\n\t\t\t\t<themekey>USGS</themekey>' \
    + '\n\t\t\t\t<themekey>U.S. Forest Service</themekey>' \
    + '\n\t\t\t\t<themekey>USFS</themekey>' \
    + '\n\t\t\t\t<themekey>Mineral Resources Program</themekey>' \
    + '\n\t\t\t\t<themekey>MRP</themekey>' \
    + '\n\t\t\t\t<themekey>Geology, Geophysics, and Geochemistry Science Center</themekey>' \
    + '\n\t\t\t\t<themekey>GGGSC</themekey>' \
    + '\n\t\t\t\t<themekey>A Shallow- to Deep- View Inside the Hydrothermally-Altered and Mineralized Silverton Caldera Complex: New Geologic Insights Gained From Modern Geophysical Interpretations project</themekey>' \
    + '\n\t\t\t\t<themekey>electric field (earth)</themekey>' \
    + '\n\t\t\t\t<themekey>impedance</themekey>\n\t\t\t\t<themekey>tipper</themekey>' \
    + '\n\t\t\t\t<themekey>apparent resistivity</themekey>\n\t\t\t\t<themekey>impedance phase</themekey>' \
    + '\n\t\t\t\t<themekey>impedance strike</themekey>\n\t\t\t\t<themekey>magnetotelluric</themekey>' \
    + '\n\t\t\t\t<themekey>MT</themekey>\n\t\t\t\t<themekey>sounding</themekey>' \
    + '\n\t\t\t\t<themekey>San Juan-Silverton Caldera Complex</themekey>\n\t\t\t\t<themekey>abandoned mine lands cleanup</themekey>' \
    + '\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>' \
    + '\n\t\t\t\t<themekey>magnetic field (earth)</themekey>' \
    + '\n\t\t\t\t<themekey>geophysics</themekey>\n\t\t\t\t<themekey>GPS measurement</themekey>' \
    + '\n\t\t\t\t<themekey>magnetotelluric surveying</themekey>\n\t\t\t</theme>\n\t\t\t<place>' \
    + '\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov</placekt>' \
    + '\n\t\t\t\t<placekey>Silverton</placekey>' \
    + '\n\t\t\t\t<placekey>Southern Rocky Mountains Volcanic Field</placekey>' \
    + '\n\t\t\t\t<placekey>' + county + ' County</placekey>\n\t\t\t</place>\n\t\t\t<place>' \
    + '\n\t\t\t\t<placekt>Common geographic areas</placekt>\n\t\t\t\t<placekey>Colorado</placekey>' \
    + '\n\t\t\t</place>\n\t\t\t</keywords>', \
    placeholder='Type something',
    #description='String:',
    layout=Layout(width='100%', height='666px'),
    disabled=False
)
print ('Keywords list created.')

Keywords list created.


### Change the text in the textbox below to relflect what should be included as the key words for all child items

Note that changing the text below at any time creates a keywords section of the metadata seen EXACTLY as it is shown below

In [116]:
# Run this cell for key word text to edit.  
# One can edit the text in place and these changes will be present in the created metadata.  
# When complete move on to the next step

display(keywords)

Textarea(value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>\n\t\t\t\t<t…

### The module below collects the file date property for use in creating the process description dates

In [117]:
# module to get file process dates for process descriptions
# Visit https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior for time format standards
# Note that get c time gives creation date.  Get m time gives creation date.
# We want cration time e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))

# Set {CollectionDate} to the station end date
strCollectionDate = enddate
print ("{CollectionDate} " + strCollectionDate)

# {CombinationDate} Get the Time stamp for the AVG file
# First we need to get the file path
avgList = glob.glob(os.path.join(mtStationPath, '*.AVG'),  recursive=True)
#print (avgList)
avgTime = time.ctime(os.path.getmtime(avgList[0]))
print ("avgTime: " + avgTime)
avgTime_obj = datetime.datetime.strptime(avgTime,'%a %b %d %H:%M:%S %Y')
strCombinationDate = datetime.datetime.strftime(avgTime_obj,'%Y%m%d')
print ("{CombinationDate} " + strCombinationDate)


# {ConversionDate} is the date of the older edi file, this is the second file in the edi list
# No that isn't it, I filtered to get the newer edi file, oh I'll just find the file 
# in an array of edi files that isn't the other one

ediAllList = glob.glob(os.path.join(mtStationPath, '*.edi'),  recursive=True)
#print (ediAllList)
for i in range (0,len(ediAllList)):
  if ediAllList[i] != ediPath:
    OriginalEDIPath = ediAllList[i]
os.path.getctime(OriginalEDIPath)
oediTime = time.ctime(os.path.getmtime(OriginalEDIPath))
print ("oediTime: " + oediTime)
oediTime_obj = datetime.datetime.strptime(oediTime,'%a %b %d %H:%M:%S %Y')
strConversionDate = datetime.datetime.strftime(oediTime_obj,'%Y%m%d')
print ("{ConversionDate} " + strConversionDate)

# {RotationDate} Get the Time stamp for the AVG file
# First we need to get the file path
dmpList = glob.glob(os.path.join(mtStationPath, '*.dmp'),  recursive=True)
#print (dmpList)
dmpTime = time.ctime(os.path.getmtime(dmpList[0]))
print ("dmpTime: " + dmpTime)
dmpTime_obj = datetime.datetime.strptime(dmpTime,'%a %b %d %H:%M:%S %Y')
strRotationDate = datetime.datetime.strftime(dmpTime_obj,'%Y%m%d')
print ("{RotationDate} " + strRotationDate)

#{HarvestDate} = final edi file 
os.path.getctime(ediPath)
filetime = time.ctime(os.path.getmtime(ediPath))
print ("Harvest Time: " + filetime)
filetime_obj = datetime.datetime.strptime(filetime,'%a %b %d %H:%M:%S %Y')
strHarvestDate = datetime.datetime.strftime(filetime_obj,'%Y%m%d')
print ("{HarvestDate} " + strHarvestDate)

{CollectionDate} 20190717
avgTime: Mon Jul  6 17:02:39 2020
{CombinationDate} 20200706
oediTime: Mon Jul  6 17:02:39 2020
{ConversionDate} 20200706
dmpTime: Mon Jul  6 17:02:39 2020
{RotationDate} 20200706
Harvest Time: Mon Jul  6 17:02:41 2020
{HarvestDate} 20200706


Entity and Attribute Values for the EDI file.  List !****FREQUENCIES****!,!****IMPEDANCE ROTATION ANGLES****!,!****IMPEDANCES****!,!****COMPUTED PARAMETERS****!

Here we load the frequencies
>!****FREQUENCIES****!

In [118]:
# Import entity and attributes - !****FREQUENCIES****! plan to break some of these individual chunks into objects/functions

# Get Range of Frequency Values in EDI File
for k in range(list_length):

 value = list_[k] 
 if value.replace(" ", "") == '>!****FREQUENCIES****!':
   startIndFrequencies = k + 2
   print ('startIndFrequencies: ' + str(startIndFrequencies))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCEROTATIONANGLES****!':
   endIndFrequencies = k - 1
   print ('endIndFrequencies: ' + str(endIndFrequencies))

frequencyData = []
fdata = []
fdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
frequencyDF = pd.DataFrame(fdata)
for j in range(startIndFrequencies,endIndFrequencies):
    fdataTemp = list_[j]
    fdataTemp = re.sub(' +', ' ',fdataTemp)
    fdataTemp = fdataTemp.split(" ")
    del fdataTemp[0]
    fdata = fdata + fdataTemp
    
print (fdata)  
fdata = np.array(fdata).astype(np.float) #convert String to floats
frequencyDF = pd.DataFrame(fdata,columns=['Frequencies'])
frequencyDF


startIndFrequencies: 286
endIndFrequencies: 292
['6.89400024E+01', '4.88300018E+01', '3.43800011E+01', '2.40200005E+01', '1.66000004E+01', '1.13299999E+01', '7.61700010E+00', '4.88299990E+00', '3.44700003E+00', '2.93000007E+00', '2.44099998E+00', '1.75800002E+00', '1.20099998E+00', '8.30100000E-01', '5.66399992E-01', '3.80899996E-01', '3.44700009E-01', '2.44100004E-01', '1.71900004E-01', '1.46500006E-01', '1.20099999E-01', '8.78999978E-02', '8.29999968E-02', '5.66000007E-02', '3.81000005E-02', '2.43999995E-02', '1.46000003E-02', '8.79999995E-03']


,Frequencies
0,68.940002
1,48.830002
2,34.380001
3,24.020000
4,16.600000
5,11.330000
6,7.617000
7,4.883000
8,3.447000
9,2.930000


In [119]:
# Now lets get the stats of the frequency data
#Make Array of Max Vallues
frequencyMax = frequencyDF[('Frequencies')].max()
print ('Max. Frequency: ' + str(frequencyMax))
frequencyMin = frequencyDF[('Frequencies')].min()
print ('Min. Frequency: ' + str(frequencyMin))

Max. Frequency: 68.9400024
Min. Frequency: 0.00879999995


Here we load the Impedance Rotation Angles
>!****IMPEDANCE ROTATION ANGLES****!

In [120]:
# Import entity and attributes - !****IMPEDANCE ROTATION ANGLES****! plan to break some of these individual chunks into objects/functions
# Get Range of IMPEDANCE ROTATION ANGLES in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCEROTATIONANGLES****!':
   startIndROT = k + 2
   print ('startIndROT: ' + str(startIndROT))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCES****!':
   endIndROT = k - 1
   print ('endIndROT: ' + str(endIndROT))

rdata = []
rdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
rotationDF = pd.DataFrame(rdata)
for j in range(startIndROT,endIndROT):
    rdataTemp = list_[j]
    rdataTemp = re.sub(' +', ' ',rdataTemp)
    rdataTemp = rdataTemp.split(" ")
    del rdataTemp[0]
    rdata = rdata + rdataTemp
    
print (rdata)  
rdata = np.array(rdata).astype(np.float) #convert String to floats
rotationDF = pd.DataFrame(rdata,columns=['ZROT'])
rotationDF

startIndROT: 295
endIndROT: 301
['0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00']


,ZROT
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [121]:
# Now lets get the stats of the rotation data

#Make Array of Max Values
rotationMax = rotationDF[('ZROT')].max()
print ('Max. ZROT: ' + str(rotationMax))

#Make Array of Min Values
rotationMin = rotationDF[('ZROT')].min()
print ('Min. ZROT: ' + str(rotationMin))

Max. ZROT: 0.0
Min. ZROT: 0.0


Here we load the impedances
>!****IMPEDANCES****!

In [122]:
# Import entity and attributes - !****IMPEDANCES****! plan to break some of these individual chunks into objects/functions
# Get Range of Impedance Values in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCES****!':
   startIndImpedances = k + 1
   print ('startIndImpedances: ' + str(startIndImpedances))
 
 if value.replace(" ", "") ==  '>!****TIPPERPARAMETERS****!':
   endIndImpedances = k - 1 
   print ('endIndImpedances: ' + str(endIndImpedances))

#Construct Array of Channel Headers   
count = 0
impedanceLabel = []
impedanceData = []
data = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
impedanceDF = pd.DataFrame(data)
for l in range (startIndImpedances,endIndImpedances): 
    if list_[l][0] == '>':
     temp = list_[l].split(" ", 1)
     #print ('List: ' + list_[l][7])
     impedanceLabel.append((temp[0].split(">"))[1])
     dataTemp = list_[l+1]
     #print ('datatemp: ' + dataTemp)
     for j in range(l+2,l+7):
      dataTemp = dataTemp + list_[j]
      #print ('datatemp: ' + dataTemp)
      dataTemp = re.sub(' +', ' ',dataTemp)
     data = dataTemp.split(" ")
     del data[0] # need to check for empty strings and delete these from the array of the string can't be converted to a float
     del data[len(data)-1] # need to check for empty strings and delete
     #print (data)
     data = np.array(data).astype(np.float) #convert String to floats
     se = pd.Series(data)
     print ((temp[0].split(">"))[1])   
     impedanceDF[((temp[0].split(">"))[1])] = se.values
    
    count = count + 1

#impedanceDF = pd.DataFrame(data, columns=(impedanceLabel))
impedanceDF
#data
#se 

startIndImpedances: 303
endIndImpedances: 398
ZXXR
ZXXI
ZXX.VAR
ZXYR
ZXYI
ZXY.VAR
ZYXR
ZYXI
ZYX.VAR
ZYYR
ZYYI
ZYY.VAR


,ZXXR,ZXXI,ZXX.VAR,ZXYR,ZXYI,ZXY.VAR,ZYXR,ZYXI,ZYX.VAR,ZYYR,ZYYI,ZYY.VAR
0,-113.593941,-64.479164,156.450027,359.047089,88.988998,308.555206,-230.697464,68.407700,113.543602,-48.088848,-153.800171,223.933960
1,32.029537,5.309176,97.507698,167.816742,68.933884,117.583656,-184.110168,-45.056229,142.158951,-25.087742,-51.372849,171.428177
2,53.889606,19.855902,46.088848,123.339516,62.312538,52.138111,-153.388168,-55.531010,158.919235,-25.142242,-25.223579,179.777740
3,40.983299,15.354822,75.335342,107.825531,52.419228,82.574799,-140.544479,-65.104210,54.298820,-19.382977,-14.436199,59.516739
4,42.459320,19.433012,54.219150,92.609383,44.314575,57.878914,-121.962143,-57.886734,49.207409,-18.765514,-11.266590,52.528885
5,41.560665,18.966614,38.459839,80.048210,37.440025,34.822803,-110.939491,-54.332043,47.910927,-21.212376,-5.347620,43.380127
6,38.968410,16.836245,44.634029,72.712044,33.823856,38.694012,-92.843849,-49.106117,86.566673,-19.463585,-7.462762,75.046150
7,26.642330,16.845495,96.451225,70.286583,35.206440,98.809486,-79.751594,-37.900577,180.043930,-15.213517,-2.896814,184.446060
8,23.362200,11.168969,5.894286,59.847065,26.308065,9.102986,-71.491409,-23.218668,8.362106,-18.090219,-7.684928,12.914225
9,24.833380,17.064321,212.498093,57.098816,28.959698,140.901016,-74.741615,-33.176739,261.076141,-14.213063,-10.896620,173.111649


In [123]:
# Now lets get the stats of the impedance data

# Make Array of Max Values
impedanceMax = []
for i in range (0,len(impedanceLabel)):
    impedanceMax.append(impedanceDF[(impedanceLabel[i])].max())
print ('Impedance Max: ' + str(impedanceMax))

# Make Array of Min Values
impedanceMin = []
for i in range (0,len(impedanceLabel)):
    impedanceMin.append(impedanceDF[(impedanceLabel[i])].min())
print ('Impedance Min: ' + str(impedanceMin))

Impedance Max: [53.8896065, 19.8559017, 212.498093, 359.047089, 88.9889984, 308.555206, -2.38664579, 68.4076996, 261.076141, 1.19574714, 0.514036, 223.93396]
Impedance Min: [-113.593941, -64.4791641, 0.0539337248, 0.524628103, 1.64940906, 0.12263076, -230.697464, -65.1042099, 0.105391033, -48.0888481, -153.800171, 0.0734180063]


In [124]:
#Make Array attributes for impedances
impAtt = ''
for i in range (0,len(impedanceLabel)):
     headerLabel = impedanceLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
     if headerLabel.find('R') != -1 and headerLabel.find('.VAR') == -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Real component of impedance ' + headerLabel[:3] + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) \
        + '</rdommin>\n\t\t\t\t\t<rdommax>' + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('I') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Imaginary component of impedance ' + headerLabel[:3] + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('.VAR') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Total variance of impedance of ' + headerLabel[:3] \
        + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms*ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\n\t\t</attr>'
        impAtt = impAtt + impAtti
   
        
print (impAtt)
ImpedanceAttributes = impAtt

Header Label: ZXXR-ZXX
Header Label: ZXXI-ZXX
Header Label: ZXX.VAR-ZXX
Header Label: ZXYR-ZXY
Header Label: ZXYI-ZXY
Header Label: ZXY.VAR-ZXY
Header Label: ZYXR-ZYX
Header Label: ZYXI-ZYX
Header Label: ZYX.VAR-ZYX
Header Label: ZYYR-ZYY
Header Label: ZYYI-ZYY
Header Label: ZYY.VAR-ZYY

		<attr>
				<attrlabl>ZXXR</attrlabl>
				<attrdef>Real component of impedance ZXX</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
					<rdommin>-113.593941</rdommin>
					<rdommax>53.8896065</rdommax>
					<attrunit>ohms</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
			<attrlabl>ZXXI</attrlabl>
				<attrdef>Imaginary component of impedance ZXX</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
					<rdommin>-64.4791641</rdommin>
					<rdommax>19.8559017</rdommax>
					<attrunit>ohms</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
				<attrlabl>ZXX.VAR</attrlabl>
				<attrdef>Total variance of impedance of ZXX</

Here we load the tipper parameters
>!****TIPPER PARAMETERS****!

In [125]:
# Import entity and attributes - !****TIPPER PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column

# Get Range of TIPPER PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****TIPPERPARAMETERS****!':
   startIndTipper = k + 1
   print ('startIndTipper: ' + str(startIndTipper))
 
 if value.replace(" ", "") ==  '>!****COMPUTEDPARAMETERS****!':
   endIndTipper = k - 1
   print ('endIndTipper: ' + str(endIndTipper))

#Construct Array of Channel Headers   
count = 0
tipperLabel = []
tipperData = []
tdata = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
tipperDF = pd.DataFrame(tdata)
for l in range(startIndTipper,endIndTipper): 
    if list_[l][0] == '>':
     ttemp = list_[l].split(" ", 1)
     #print (ttemp)
     tipperLabel.append((ttemp[0].split(">"))[1])
     tdataTemp = list_[l+1]
     for j in range(l+2,l+7):
      tdataTemp = tdataTemp + list_[j]
      tdataTemp = re.sub(' +', ' ',tdataTemp)
      tdata = tdataTemp.split(" ")
     #print (tdata)
     del tdata[0]
     del tdata[len(tdata)-1] # need to check for empty strings and delete
     tdata = np.array(tdata).astype(np.float) #convert String to floats
     te = pd.Series(tdata)
     print ((ttemp[0].split(">"))[1])   
     tipperDF[((ttemp[0].split(">"))[1])] = te.values
    
    count = count + 1

#tipperDF = pd.DataFrame(tdata, columns=(tipperLabel))
tipperDF
#tdata
#te 

startIndTipper: 400
endIndTipper: 447
TXR.EXP
TXI.EXP
TXVAR.EXP
TYR.EXP
TYI.EXP
TYVAR.EXP


,TXR.EXP,TXI.EXP,TXVAR.EXP,TYR.EXP,TYI.EXP,TYVAR.EXP
0,2.002357,0.718964,0.015884,-2.015301,-0.240124,0.031326
1,0.970383,0.245906,0.022319,-0.822739,-0.152433,0.026914
2,0.717950,0.303751,0.028908,-0.475684,-0.157904,0.032702
3,0.560588,0.240824,0.017336,-0.337413,-0.116622,0.019002
4,0.387916,0.259227,0.017876,-0.232353,-0.164605,0.019083
5,0.203304,0.266841,0.016128,-0.094247,-0.126742,0.014603
6,0.185662,0.285162,0.032523,-0.045805,-0.105015,0.028195
7,0.239734,0.293569,0.073903,-0.026165,-0.079745,0.075710
8,-0.111079,0.081635,0.001212,0.010499,-0.106929,0.001872
9,-0.033604,0.406500,0.121069,0.021031,0.025081,0.080277


In [126]:
# Now lets get the stats of the tipper data

# Make Array of Max Values
tipperMax = []
for i in range (0,len(tipperLabel)):
    tipperMax.append(tipperDF[(tipperLabel[i])].max())
print ('Tipper Max: ' + str(tipperMax))    

# Make Array of Min Values
tipperMin = []
for i in range (0,len(tipperLabel)):
    tipperMin.append(tipperDF[(tipperLabel[i])].min())
print ('Tipper Min: ' + str(tipperMin))

Tipper Max: [2.00235653, 0.718963802, 0.121069193, 0.200346068, 0.232504264, 0.0802773014]
Tipper Min: [-0.262561232, -0.253840804, 0.000939624093, -2.01530123, -0.279473722, 0.00138717995]


In [127]:
#Make Array attributes for tipper parameters
tipAtt = ''
for i in range (0,len(tipperLabel)):
     headerLabel = tipperLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
#     if headerLabel.find('R') != -1 and headerLabel.find('.VAR') == -1 and headerLabel.find('.EXP') == -1:
     if headerLabel.find('R') != -1 and headerLabel.find('VAR') == -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Real part of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('I') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Imaginary part of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('.VAR') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Total variance of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('VAR') != -1 and headerLabel.find('.VAR') == -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Total variance of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
        
print (tipAtt)
TipperAttributes = tipAtt

Header Label: TXR.EXP-TXR
Header Label: TXI.EXP-TXI
Header Label: TXVAR.EXP-TXV
Header Label: TYR.EXP-TYR
Header Label: TYI.EXP-TYI
Header Label: TYVAR.EXP-TYV
		<attr>
			<attrlabl>TXR.EXP</attrlabl>
			<attrdef>Real part of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>-0.262561232</rdommin>
			<rdommax>2.00235653</rdommax>
			<attrunit>unitless</attrunit>
			</rdom>
			</attrdomv>
		</attr>
		<attr>
			<attrlabl>TXI.EXP</attrlabl>
			<attrdef>Imaginary part of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>-0.253840804</rdommin>
			<rdommax>0.718963802</rdommax>
			<attrunit>unitless</attrunit>
			</rdom>
			</attrdomv>
		</attr>
		<attr>
			<attrlabl>TXVAR.EXP</attrlabl>
			<attrdef>Total variance of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>0.000939624093</

Here we load the computed parameters
>!****COMPUTED PARAMETERS****!

In [128]:
# Import entity and attributes - !****COMPUTED PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column
# Get Range of COMPUTED PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****COMPUTEDPARAMETERS****!':
   startIndPar = k + 1
   print ('startIndPar: ' + str(startIndPar))
 
 if value.replace(" ", "") ==  '>END':
   endIndPar = k - 1
   print ('endIndPar: ' + str(endIndPar))

#Construct Array of Channel Headers   
count = -1
currentLabel = ''
parLabel = []
parData = []
pdata = []
ampIndex = 1
noiseIndex = 1
cohIndex = 1
epIndex = 1
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
parDF = pd.DataFrame(pdata)
for l in range(startIndPar,endIndPar): 
    if list_[l][0] == '>':
      ptemp = list_[l].split(" ", 1)
     #print ("ptemp:\n" + str(ptemp))
      if ptemp[0].split(">")[1] == 'SIGAMP':  
       parLabel.append((ptemp[0].split(">"))[1] + str(ampIndex))
       ampIndex = ampIndex + 1  
      elif ptemp[0].split(">")[1] == 'SIGNOISE':  
       parLabel.append((ptemp[0].split(">"))[1] + str(noiseIndex))
       noiseIndex = noiseIndex + 1 
      elif ptemp[0].split(">")[1] == 'COH':  
       parLabel.append((ptemp[0].split(">"))[1] + str(cohIndex))
       cohIndex = cohIndex + 1
      elif ptemp[0].split(">")[1] == 'EPREDCOH':  
       parLabel.append((ptemp[0].split(">"))[1] + str(epIndex))
       epIndex = epIndex + 1
      else:
        parLabel.append((ptemp[0].split(">"))[1])  
        
for l in range(startIndPar,endIndPar): 
    if list_[l][0] == '>':
     count = count + 1
     pdataTemp = list_[l+1]
     for j in range(l+2,l+7):
       pdataTemp = pdataTemp + list_[j]
       pdataTemp = re.sub(' +', ' ',pdataTemp)
       pdata = pdataTemp.split(" ")
       #print ("pdata:\n" + str(pdata))
       del pdata[0]
       del pdata[len(pdata)-1] # need to check for empty strings and delete
       pdata = np.array(pdata).astype(np.float) #convert String to floats
       pe = pd.Series(pdata)
       #print (parLabel[count]  + ':' + str(l) )
       currentLabel = parLabel[count]
    parDF[currentLabel] = pe.values
    

#pdataTemp
#parLabel
parDF
#pdata
#pe
#te.values

startIndPar: 449
endIndPar: 768


,RHOROT,RHOXX,RHOXX.ERR,RHOXY,RHOXY.ERR,RHOYX,RHOYX.ERR,RHOYY,RHOYY.ERR,PHSXX,...,SIGAMP1,SIGAMP2,SIGAMP3,SIGAMP4,SIGAMP5,SIGNOISE1,SIGNOISE2,SIGNOISE3,SIGNOISE4,SIGNOISE5
0,0.0,49.495636,0.058812,396.965027,0.029165,167.974854,0.027197,75.332260,0.057034,-150.419510,...,0.006212,0.004423,0.021568,2.387025,1.894402,1.0,1.0,0.949106,0.953960,0.946949
1,0.0,4.317340,0.186797,134.811951,0.036709,147.149765,0.038634,13.387526,0.140653,9.411700,...,0.003055,0.002782,0.005257,0.601590,0.701782,1.0,1.0,0.401119,0.798291,0.783899
2,0.0,19.187588,0.072600,111.084862,0.032092,154.808746,0.047461,7.378483,0.231224,20.226578,...,0.003260,0.003065,0.004679,0.545329,0.648476,1.0,1.0,0.270395,0.912045,0.785527
3,0.0,15.948387,0.121802,119.684601,0.046550,199.761108,0.029218,4.863477,0.196047,20.538988,...,0.003270,0.003124,0.003337,0.472533,0.554091,1.0,1.0,0.359541,0.853574,0.923244
4,0.0,26.270309,0.096848,126.991310,0.045511,219.585999,0.031912,5.772055,0.203367,24.592878,...,0.003143,0.003042,0.002495,0.410765,0.467939,1.0,1.0,0.309807,0.913673,0.939628
5,0.0,36.840626,0.083373,137.854752,0.041012,269.365265,0.034414,8.447695,0.184911,24.530047,...,0.003508,0.003687,0.002119,0.422235,0.481077,1.0,1.0,0.274436,0.949675,0.951707
6,0.0,47.315117,0.096659,168.861618,0.047639,289.651855,0.054406,11.409320,0.255237,23.366688,...,0.003683,0.003955,0.002616,0.411616,0.443583,1.0,1.0,0.203096,0.952007,0.919852
7,0.0,40.695656,0.191354,253.110672,0.077661,319.343445,0.093329,9.823577,0.538589,32.304474,...,0.003211,0.003173,0.003022,0.315672,0.333763,1.0,1.0,0.147268,0.893594,0.822322
8,0.0,38.905613,0.057582,247.971298,0.028345,327.828735,0.023627,22.414514,0.112292,25.551468,...,0.002703,0.002175,0.001481,0.196508,0.245611,1.0,1.0,0.078358,0.740600,0.764431
9,0.0,61.971867,0.297131,279.791046,0.113869,456.450867,0.121354,21.894028,0.451198,34.495003,...,0.003961,0.004864,0.003942,0.403742,0.404057,1.0,1.0,0.177534,0.865851,0.835440


In [129]:
# Now lets get the stats of the computed parameters

# Make Array of Max Values
parMax = []
for i in range (0,len(parLabel)):
 parMax.append(parDF[(parLabel[i])].max())
 print ('Computed Pararmeters Max of ' + str(parLabel[i]) + ': ' + str(parMax[i]))    

# Make Array of Min Values
parMin = []
for i in range (0,len(parLabel)):
 parMin.append(parDF[(parLabel[i])].min())
 print ('Computed Pararmeters Min: ' + str(parMin))

for i in range (0,len(parLabel)):
    print ('Header Index ' + str(parLabel[i]) + ':' + str(i))

Computed Pararmeters Max of RHOROT: 0.0
Computed Pararmeters Max of RHOXX: 242.446579
Computed Pararmeters Max of RHOXX.ERR: 0.297130525
Computed Pararmeters Max of RHOXY: 396.965027
Computed Pararmeters Max of RHOXY.ERR: 0.187654674
Computed Pararmeters Max of RHOYX: 899.211853
Computed Pararmeters Max of RHOYX.ERR: 0.121353865
Computed Pararmeters Max of RHOYY: 76.4581223
Computed Pararmeters Max of RHOYY.ERR: 1.33401942
Computed Pararmeters Max of PHSXX: 69.4820862
Computed Pararmeters Max of PHSXX.ERR: 19.600584
Computed Pararmeters Max of PHSXY: 72.4439392
Computed Pararmeters Max of PHSXY.ERR: 12.3788748
Computed Pararmeters Max of PHSYX: 163.483551
Computed Pararmeters Max of PHSYX.ERR: 8.00525856
Computed Pararmeters Max of PHSYY: 169.566101
Computed Pararmeters Max of PHSYY.ERR: 88.0002518
Computed Pararmeters Max of TIPMAG: 2.94031286
Computed Pararmeters Max of TIPMAG.ERR: 1.00000003e+32
Computed Pararmeters Max of TIPPHS: 159.628265
Computed Pararmeters Max of TIPPHS.ERR: 1

In [130]:
# assign variables for xml file find and replace
RHOROTmax = parMax[0]
RHOROTmin = parMin[0]
print ('RHOROTmax = ' + str(RHOROTmax))
print ('RHOROTmin = ' + str(RHOROTmin))
RHOXXmax = parMax[1]
RHOXXmin = parMin[1]
print ('RHOXXmax = ' + str(RHOXXmax))
print ('RHOXXmin = ' + str(RHOXXmin))
PHSXXmax = parMax[9]
PHSXXmin = parMin[9]
RHOXXERRmax = parMax[2]
RHOXXERRmin = parMin[2]
PHSXXERRmax = parMax[10]
PHSXXERRmin = parMin[10]
RHOXYmax = parMax[3]
RHOXYmin = parMin[3]
PHSXYmax = parMax[11]
PHSXYmin = parMin[11]
RHOXYERRmax = parMax[4]
RHOXYERRmin = parMin[4]
PHSXYERRmax = parMax[12]
PHSXYERRmin = parMin[12]
RHOYXmax = parMax[5]
RHOYXmin = parMin[5]
PHSYXmax = parMax[13]
PHSYXmin = parMin[13]
RHOYXERRmax = parMax[6]
RHOYXERRmin = parMin[6]
PHSYXERRmax = parMax[14]
PHSYXERRmin = parMin[14]
RHOYYmax = parMax[7]
RHOYYmin = parMin[7]
PHSYYmax = parMax[15]
PHSYYmin = parMin[15]
RHOYYERRmax = parMax[8]
RHOYYERRmin = parMin[8]
PHSYYERRmax = parMax[16]
PHSYYERRmin = parMin[16]
TIPMAGmax = parMax[17]
TIPMAGmin = parMin[17]
print ('TIPMAGmin = ' + str(TIPMAGmin))
TIPPHSmax = parMax[19]
TIPPHSmin = parMin[19]
TIPMAGERRmax = parMax[18]
TIPMAGERRmin = parMin[18]
TIPPHSERRmax = parMax[20]
TIPPHSERRmin = parMin[20]
ZSTRIKEmax = parMax[21]
ZSTRIKEmin = parMin[21]
ZSKEWmax = parMax[22]
ZSKEWmin = parMin[22]
TSTRIKEmax = parMax[23]
TSTRIKEmin = parMin[23]
COH1max = parMax[24]
COH1min = parMin[24]
COH2max = parMax[25]
COH2min = parMin[25]
COH3max = parMax[26]
COH3min = parMin[26]
COH4max = parMax[27]
COH4min = parMin[27]
EPREDCOH1max = parMax[28]
EPREDCOH1min = parMin[28]
EPREDCOH2max = parMax[29]
EPREDCOH2min = parMin[29]
SIGAMP1max = parMax[30]
SIGAMP1min = parMin[30]
SIGAMP2max = parMax[31]
SIGAMP2min = parMin[31]
SIGAMP3max = parMax[32]
SIGAMP3min = parMin[32]
SIGAMP4max = parMax[33]
SIGAMP4min = parMin[33]
SIGAMP5max = parMax[34]
SIGAMP5min = parMin[34]
SIGNOISE1max = parMax[35]
SIGNOISE1min = parMin[35]
SIGNOISE2max = parMax[36]
SIGNOISE2min = parMin[36]
SIGNOISE3max = parMax[37]
SIGNOISE3min = parMin[37]
SIGNOISE4max = parMax[38]
SIGNOISE4min = parMin[38]
SIGNOISE5max = parMax[39]
SIGNOISE5min = parMin[39]

RHOROTmax = 0.0
RHOROTmin = 0.0
RHOXXmax = 242.446579
RHOXXmin = 4.3173399
TIPMAGmin = 0.0800671428


## Now lets get the range of values from the RSP values

## Now the raw Binary File Listing - this can be T files or W files
We will need to figure out the best way of filtering on thise - may need to build array and then delete AVG, dmp and edi file.

These are listed in the edi file as well but they are not all there.  

    ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 

- Which files need to be included in the data release?
- What is the best way to get this listing?

In [131]:
#First Get the list of RSP files
rspList = glob.glob(os.path.join(mtStationPath, '*.RSP'),  recursive=True)
#rspList
ediFile = str(ediPathArray[len(ediPathArray)-1])
fileListing = ''
fileListing = fileListing + '\t\t\t\t\t\t' + ediFile + '\n' #start the file listing with the main EDI file
rspFileListing = []
for i in range(len(rspList)):
    splitRspList = rspList[i].split('\\')
    fileListing = fileListing + '\t\t\t\t\t\t' + splitRspList[len(splitRspList) - 1] + '\n'
    rspFileListing.append(splitRspList[len(splitRspList) - 1])
print (fileListing)


						USA-Colorado-Silverton_Caldera-2019-MTAG01.edi
						BF4-9820.RSP
						BF4-9821.RSP
						BF7-8004.RSP
						EF-9515X.RSP
						EF-9515Y.RSP



In [132]:
#Now finally add the processed ASCII text files to the list
txtList = glob.glob(os.path.join(mtStationPath, '*.txt'),  recursive=True)
txtListFormatted = []
#txtList
print ('Text File Only Listing:\n')
for i in range(len(txtList)):
  splitTxtList = txtList[i].split('\\')
  txtListFormatted.append(splitTxtList[len(splitTxtList) - 1])
  #print (txtListFormatted[i])  
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTxtList[len(splitTxtList) - 1] + '\n'

#print ('File Listing:\n' + fileListing)
#Remove readme file from list.  May need to search an remove if it does not come up first in alphabetical order?'
del txtListFormatted[0]
print (txtListFormatted)

Text File Only Listing:

['USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_High_01.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_High_02.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_01.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_02.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_03.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_04.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_05.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Mid_01.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Mid_02.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Mid_03.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Mid_04.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-TS_High_01.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-TS_High_02.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-TS_Low_01.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-TS_Low_02.txt', 'USA-Colorado-Silverton_Caldera-2019-MTAG01-TS_Low_03.txt', 'USA-Color

In [133]:
# Create the Entity and Attributes for the .txt? listing (Note: for each station, replace sciencebase link with correct link)
txtEandA = ''
#As before, we read in the list of files

for i in range(len(txtListFormatted)):
    if txtListFormatted[i].find('-BP') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + ' </enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for raw cross power files.</eaover> '\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw cross power files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if txtListFormatted[i].find('-FC') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File. ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary Fourier coefficient files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw Fourier coefficient files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for Fourier coefficient binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for Fourier coefficient binary content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if txtListFormatted[i].find('-SD') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for synchronous detection cross power files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for synchronous detection cross power files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if txtListFormatted[i].find('-TS') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary time series files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for binary time series files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for time series binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Header description and settings for time series binary content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
#Add the final .txt listing for the readme file.  
#Be sure to comment this out if there is no readme.txt file included with the data release

#readmeEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File readme.txt</enttypl>'\
readmeEandA = '\t\t<overview>\n\t\t<eaover>Text File, readme.txt, is a Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</eaover>'\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\n\t\t\t<enttypd>Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</enttypd>'\
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>'\
#        + '\n\t\t</enttyp>\n\t\t</detailed>\n'
    
txtEandA = txtEandA + readmeEandA
print (txtEandA)
ALL_EandA = '' #'<eainfo>\n'   
ALL_EandA = ALL_EandA + txtEandA

		<overview>
			<eaover>Text File, USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_High_01.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5efa5e5782ced62aaaee1e9b</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_High_02.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5efa5e5782ced62aaaee1e9b</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_01.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5efa5e5782ced62aaaee1e9b</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Colorado-Silverton_Caldera-2019-MTAG01-BP_Low_02.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https

# Now get values and stats on the .RSP files that are listed to add to the Ent. and Att. information

### These files are all fixed width format (666) but it looks like the BFS*.RSP and EF*.RSP are slightly different beasts that have different formats

In [134]:
# Load the BFS*.RSP files into pandasand create entities and attributes
allRspEandP = ''
strFreqMaxBSP = ''
strAmpMaxBSP = ''
strGammaMaxBSP = ''
strFreqMinBSP = ''
strAmpMinBSP = ''
strGammaMinBSP = ''

strFreqMaxESP = ''
strFreqMinESP = ''
strAmp1MaxESP = ''
strAmp1MinESP = ''
strAmp2MaxESP = ''
strAmp2MinESP = ''
strAmp3MaxESP = ''
strAmp3MinESP = ''
strAmp4MaxESP = ''
strAmp4MinESP = ''
strPhz1MaxESP = ''
strPhz1MinESP = ''
strPhz2MaxESP = ''
strPhz2MinESP = ''
strPhz3MaxESP = ''
strPhz3MinESP = ''
strPhz4MaxESP = ''
strPhz4MinESP = ''

intNumberBSF = 0
intNumberEFF = 0
for i in range(len(rspList)):
#for i in range(3):
    if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        bfRSP = pd.read_fwf(rspList[i], widths=[6,6,6], skiprows=5, parse_dates=True).rename(columns={'31':'Freq', '1':'Amp', 'Unnamed: 2':'Gamma'})
#print (rspList[1])
        #print (bfRSP)    
    
# Now lets get the stats of the bfRSP data

# Make Array of Min and Max Values
        FreqMaxRSP = bfRSP['Freq'].max()
        strFreqMaxBSP = str(FreqMaxRSP)
        AmpMaxRSP = bfRSP['Amp'].max()
        strAmpMaxBSP = str(AmpMaxRSP)
        GammaMaxRSP = bfRSP['Gamma'].max()
        strGammaMaxBSP = str(GammaMaxRSP)
        FreqMinRSP = bfRSP['Freq'].min()
        strFreqMinBSP = str(FreqMinRSP)
        AmpMinRSP = bfRSP['Amp'].min()
        strAmpMinBSP = str(AmpMinRSP)
        GammaMinRSP = bfRSP['Gamma'].min()
        strGammaMinBSP = str(GammaMinRSP)
# now print RSP entity and attribute    
        bspEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts/Gamma</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmpMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmpMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts/Gamma</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strGammaMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strGammaMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        #print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  bspEandA + '\n'
    if rspList[i].find('EF') > 0:
        #print ('i Loop = ' + str(i) + ' EF File Found')
        efRSP = pd.read_fwf(rspList[i], widths=[6,6,6,6,6,6,6,6,6], skiprows=5, parse_dates=True)\
        .rename(columns={'42':'Freq', '4':'Amp1', 'Low F':'Phz1', 'requen':'Amp2', 'cy':'Phz2', '10Hz':'Amp3', 'Out':'Phz3',\
        'Unnamed: 7':'Amp4', 'Unnamed: 8':'Phz4'})
#Delete headers from data to remove all strings from Pandas Data Frame

        efRSP = efRSP[efRSP.Amp2 != 'Freque']
        efRSP = efRSP[efRSP.Phz1 != 'Low F']
        #print (str(efRSP))   
# Make Array of Min and Max Values
        strFreqMaxESP = str(efRSP['Freq'].max())
        strFreqMinESP = str(efRSP['Freq'].min())
        strAmp1MaxESP = str(efRSP['Amp1'].max())
        strAmp1MinESP = str(efRSP['Amp1'].min())
        strAmp2MaxESP = str(efRSP['Amp2'].max())
        strAmp2MinESP = str(efRSP['Amp2'].min())
        strAmp3MaxESP = str(efRSP['Amp3'].max())
        strAmp3MinESP = str(efRSP['Amp3'].min())
        strAmp4MaxESP = str(efRSP['Amp4'].max())
        strAmp4MinESP = str(efRSP['Amp4'].min())
        strPhz1MaxESP = str(efRSP['Phz1'].max())
        print ('strPhz1MaxESP = ' + (strPhz1MaxESP))
        strPhz1MinESP = str(efRSP['Phz1'].min())
        strPhz2MaxESP = str(efRSP['Phz2'].max())
        strPhz2MinESP = str(efRSP['Phz2'].min())
        strPhz3MaxESP = str(efRSP['Phz3'].max())
        print ('strPhz3MaxESP = ' + strPhz3MaxESP)
        strPhz3MinESP = str(efRSP['Phz3'].min())
        strPhz4MaxESP = str(efRSP['Phz4'].max())
        print ('strPhz4MaxESP = ' + strPhz4MaxESP)
        strPhz4MinESP = str(efRSP['Phz4'].min())
        espEandA = '\t<detailed>\n\t\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmp1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmp1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strPhz1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strPhz1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        #print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  espEandA + '\n' 
        #print (str(allRspEandP)) 

     
EandA = allRspEandP # [-:2] # this removes the last characters of the string to get rid of the last line return.  
ALL_EandA = ALL_EandA +  allRspEandP


KeyError: 'Freq'

## Now let's work on the SD mode files

### Overview of SD files,

The prefix file naming convention for the SD mode AMT non-transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run number (first run is A1, second run is A2, third run is A3, etc.)

The suffix file naming convention for the SD mode AMT non-transmitter files are:
SD6 sample frequencies are always 79, 90, 100, 150, 210, 270, 340, 460, 580 Hertz
SD7 sample frequencies are always 340, 460, 580, 720, 885, 1170 Hertz
SD8 sample frequencies are always 1170, 1500, 1870, 2200, 2730, 3550, 4900, 6500, 9000 Hertz
SD9 sample frequencies are always 6500, 9000, 11590, 15290, 19500, 23370 Hertz

The prefix file naming convention for the SD mode AMT transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run "number" (first run is AA, second run is AB, third run is AC, etc.)

The suffix file naming convention for the SD mode AMT transmitter files are:
SDA sample frequency is always 960 Hertz
SDB sample frequency is always 1200 Hertz
SDC sample frequency is always 1870 Hertz
SDD sample frequency is always 2420 Hertz
SDE sample frequency is always 2730 Hertz
SDF sample frequency is always 3550 Hertz
SDG sample frequency is always 5210 Hertz
SDH sample frequency is always 6850 Hertz
SDI sample frequency is always 11590 Hertz
SDJ sample frequency is always 15920 Hertz
SDK sample frequency is always 23370 Hertz

The file format of the SD mode files are described in each of the text files (last block of information within each file). It is the same data format regardless of whether or not a transmitter was used to collect the data. We must have been looking at the text file for the FC (Fourier Coefficient) files in your office, so I was confused that I hadn't described the data format (it is there in the text files for all SD files).



In [1169]:
#First get a listing of the SD mode files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdaList
sdFileList = ''
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  sdFileList = sdFileList + splitSdList[len(splitSdList) - 1] + '\n'

#print ('File ListingfileListing:\n' + fileListing)
print ('SD file listing:\n' + sdFileList)


SD file listing:
SF06A1.SD6
SF06A1.SD7
SF06A1.SD8
SF06A1.SD9
SF06A11.SD6
SF06A11.SD7
SF06A11.SD8
SF06A11.SD9
SF06A12.SD6
SF06A12.SD7
SF06A12.SD8
SF06A12.SD9
SF06A13.SD6
SF06A13.SD7
SF06A13.SD8
SF06A13.SD9
SF06A14.SD6
SF06A14.SD7
SF06A14.SD8
SF06A14.SD9
SF06A15.SD6
SF06A15.SD7
SF06A15.SD8
SF06A15.SD9
SF06A16.SD6
SF06A16.SD7
SF06A16.SD8
SF06A16.SD9



In [1170]:
#listing of the Fourier Coefficient files (FC) files.
#These are Binary so we don't need to come up with the range of the values.
fcList = glob.glob(os.path.join(mtStationPath, '*.fc?'),  recursive=True)
#fcList
for i in range(len(fcList)):
  splitFcList = fcList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitFcList[len(splitFcList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2019-AMTSF06.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						amtsf06txt.txt
						readme.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_04.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_05.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_06.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_07.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF0

In [1171]:
# Create the Entity and Attributes for the FC listing
fcEandAi = ''
fcEandA = ''
for i in range(len(fcList)):
    splitFcList = fcList[i].split('\\')
#    fcEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitFcList[len(splitFcList) - 1] + '</enttypl>\n' \
    fcEandAi = '\t\t<overview>\n\t\t<eaover>Binary File, ' + splitFcList[len(splitFcList) - 1] + ' is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
    fcEandA = fcEandA + fcEandAi

ALL_EandA = ALL_EandA + fcEandA    
print (fcEandA)        
        
        
        
        

		<overview>
		<eaover>Binary File, SF06A1.FC6 is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SF06A1.FC7 is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SF06A1.FC8 is a raw Fourier Coefficient Binary Data File. The header

In [1172]:
#listing of the Time Series TS? files
#Now finally add the processed ASCII text files to the list
tsList = glob.glob(os.path.join(mtStationPath, '*.ts?'),  recursive=True)
#tsList
for i in range(len(tsList)):
  splitTsList = tsList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTsList[len(splitTsList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2019-AMTSF06.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						amtsf06txt.txt
						readme.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_04.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_05.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_06.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_07.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF0

In [1173]:
# Create the Entity and Attributes for the Time series .TS? listing

tsEandAi = ''
tsEandA = ''
for i in range(len(tsList)):
    splitTsList = tsList[i].split('\\')
#    tsEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitTsList[len(splitTsList) - 1] + '</enttypl>\n' \
    tsEandAi = '\t\t<overview>\n\t\t<eaover>Binary File, ' + splitTsList[len(splitTsList) - 1] + ' is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>' \
#        + '\n\t\t</enttyp>\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t\t</attrdomv>' \
#        + '\t\t</attr>\n\t</detailed>\n'
    tsEandA = tsEandA + tsEandAi

ALL_EandA = ALL_EandA + tsEandA
print (tsEandA)

		<overview>
		<eaover>Binary File, SF06A1.TS1 is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SF06A2.TS1 is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>



In [1174]:
#listing of the BP? files
#Now finally add the processed ASCII text files to the list
bpList = glob.glob(os.path.join(mtStationPath, '*.bp?'),  recursive=True)
#bpList
for i in range(len(bpList)):
  splitBpList = bpList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitBpList[len(splitBpList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Colorado-Silverton_Caldera-2019-AMTSF06.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						amtsf06txt.txt
						readme.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-BP_High_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_04.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_05.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_06.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC6_07.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_01.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_02.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF06-FC7_03.txt
						USA-Colorado-Silverton_Caldera-2019-AMTSF0

In [1175]:
# Create the Entity and Attributes for the .BP? listing

#As before, we read in the list of files

print (bpList)


for i in range(len(bpList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfBP = pd.read_fwf(bpList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

bpEandAi = ''
bpEandA = ''
for i in range(len(bpList)):
    splitBpList = bpList[i].split('\\')
#    bpEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitBpList[len(splitBpList) - 1] + '</enttypl>\n' \
    bpEandAi = '\t\t<overview>\n\t\t<eaover>Text File, ' + splitBpList[len(splitBpList) - 1] + ' is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
#        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
    bpEandA = bpEandA + bpEandAi
    
ALL_EandA = ALL_EandA +  bpEandA
print (bpEandA)

['C:\\Temp\\MT1-DR\\Animas-AMT\\amtsf06\\SF06A1.BP1', 'C:\\Temp\\MT1-DR\\Animas-AMT\\amtsf06\\SF06A2.BP1']
		<overview>
		<eaover>Text File, SF06A1.BP1 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>
		<overview>
		<eaover>Text File, SF06A2.BP1 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e6a6d16e4b01d509260e769</eadetcit>
		</overview>



In [1176]:
#listing of the SD? files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdList
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  #fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  print ('SD? File:\n' + splitSdList[len(splitSdList) - 1])


SD? File:
SF06A1.SD6
SD? File:
SF06A1.SD7
SD? File:
SF06A1.SD8
SD? File:
SF06A1.SD9
SD? File:
SF06A11.SD6
SD? File:
SF06A11.SD7
SD? File:
SF06A11.SD8
SD? File:
SF06A11.SD9
SD? File:
SF06A12.SD6
SD? File:
SF06A12.SD7
SD? File:
SF06A12.SD8
SD? File:
SF06A12.SD9
SD? File:
SF06A13.SD6
SD? File:
SF06A13.SD7
SD? File:
SF06A13.SD8
SD? File:
SF06A13.SD9
SD? File:
SF06A14.SD6
SD? File:
SF06A14.SD7
SD? File:
SF06A14.SD8
SD? File:
SF06A14.SD9
SD? File:
SF06A15.SD6
SD? File:
SF06A15.SD7
SD? File:
SF06A15.SD8
SD? File:
SF06A15.SD9
SD? File:
SF06A16.SD6
SD? File:
SF06A16.SD7
SD? File:
SF06A16.SD8
SD? File:
SF06A16.SD9


In [1177]:
#SD files Entity and Attributes
for i in range(len(sdList)):
    print (sdList[i])
    file = open(sdList[i],"r")
    Content = file.read() 
    CoList = Content.split("\n")
    Counter = 0
    for j in CoList: 
        if j: 
            Counter += 1
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
    print ("Count = " + str(Counter))
    if Counter > 27:
        dfBP = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
        print ("File " + sdList[i] + " is an aborted run file")
        #Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
   # print (dfSD)        

##for i in range(len(sdList)):
    #for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        ##dfBP = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

sdEandA = ''
sdEandAi = ''

for i in range(len(sdList)):
    print (sdList[i])
    file = open(sdList[i],"r")
    Content = file.read() 
    CoList = Content.split("\n")
    Counter = 0
    for j in CoList: 
        if j: 
            Counter += 1
    
    if Counter > 27:
        splitSdList = sdList[i].split('\\')
#    sdEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitSdList[len(splitSdList) - 1] + '</enttypl>\n' \
        sdEandAi = '\t\t<overview>\n\t\t<eaover>Text File, ' + splitSdList[len(splitSdList) - 1] + ' is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
        + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
#        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
        sdEandA = sdEandA + sdEandAi
    if Counter < 28:
        splitSdList = sdList[i].split('\\')
#    sdEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitSdList[len(splitSdList) - 1] + '</enttypl>\n' \
        sdEandAi = '\t\t<overview>\n\t\t<eaover>Text File, ' + splitSdList[len(splitSdList) - 1] + ' is a raw Cross Power Data File from an aborted run. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
#        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
        sdEandA = sdEandA + sdEandAi
    
ALL_EandA = ALL_EandA +  sdEandA
#print (len(sdList))
#print ((sdList))
print (sdEandA)

C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD6
Count = 447
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD6 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD7
Count = 279
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD7 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD8
Count = 363
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD8 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD9
Count = 195
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A1.SD9 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD6
Count = 97
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD6 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD7
Count = 69
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD7 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD8
Count = 83
File C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD8 is an aborted run file
C:\Temp\MT1-DR\Animas-AMT\amtsf06\SF06A11.SD9
Count = 55
File C:\Temp\MT1-DR\Animas-AMT

# Populate Metadata Template

In [1178]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(mtMetaDataTemplatePath, mtMetaDataTemplateName)
print ('Metadata path: ' + mtMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\Temp\MT1-DR\SquamataAMT

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '\t\t\t\t<origin>Reitman, J.</origin>\n', '\t\t\t\t<origin>Rodriguez, B.D.</origin>\n', '\t\t\t\t<origin>Brown, P.J.</origin>\t\t\t\t\t\t\t\t\n', '\t\t\t\t<pubdate>2020</pubdate>\n', '\t\t\t\t<title>{title}</title>\n', '\t\t\t\t<edition>1</edition>\n', '\t\t\t\t<geoform>ASCII and Binary Digital Data</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators: Reitman, J., http://orcid.org/0000-0003-3551-9884; Rodriguez, B.D., http://orcid.org/0000-0002-2263-611X; Brown, P.J., http://orcid.org/0000-0002-2415-7462</othercit>\n', '\t\t\t\t<onlink>https://doi.org/10.5066/P9F68AQP</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', '\t\t\t<abstr

In [1179]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
newMetaDataContent = metaDataContent
splitFileName = ediList[0].split('.')
myfilename = splitFileName[0] + '.xml'
xmlFile = open(myfilename,"w+")
print(myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', citTit + '; Station ' + strChildID)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', purposeClean)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', descriptionClean)
    
    if lineString.find('{BeginFileListingHere}'):
     lineString = lineString.replace('{BeginFileListingHere}', fileListing)
    
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', keywords.value)
    
    if lineString.find('{CollectionDate}'):
     lineString = lineString.replace('{CollectionDate}', strCollectionDate)
    
    if lineString.find('{CombinationDate}'):
     lineString = lineString.replace('{CombinationDate}', strCombinationDate)

    if lineString.find('{ConversionDate}'):
     lineString = lineString.replace('{ConversionDate}', strConversionDate)

    if lineString.find('{RotationDate}'):
     lineString = lineString.replace('{RotationDate}', strRotationDate)
    
    if lineString.find('{HarvestDate}'):
     lineString = lineString.replace('{HarvestDate}', strHarvestDate)
   
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLon}', sitLon)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLat}', sitLat)
    
    if lineString.find('{EandA}'):
     lineString = lineString.replace('{EandA}', ALL_EandA)
    
    if lineString.find('{frequencyMin}'):
     lineString = lineString.replace('{frequencyMin}', str(frequencyMin))
    
    if lineString.find('{frequencyMax}'):
     lineString = lineString.replace('{frequencyMax}', str(frequencyMax))
    
    if lineString.find('{rotationMax}'):
     lineString = lineString.replace('{rotationMax}', str(rotationMax))
    
    if lineString.find('{rotationMin}'):
     lineString = lineString.replace('{rotationMin}', str(rotationMin)) 
    
    if lineString.find('{EdiFileName}'):
     lineString = lineString.replace('{EdiFileName}', EdiFileName)
    
    if lineString.find('{RHOROTmax}'):
     lineString = lineString.replace('{RHOROTmax}', str(RHOROTmax))
    
    if lineString.find('{RHOROTmin}'):
     lineString = lineString.replace('{RHOROTmin}', str(RHOROTmin))
    
    if lineString.find('{RHOXXmax}'):
     lineString = lineString.replace('{RHOXXmax}', str(RHOXXmax))
    
    if lineString.find('{RHOXXmin}'):
     lineString = lineString.replace('{RHOXXmin}', str(RHOXXmin)) 
    
    if lineString.find('{PHSXXmax}'):
     lineString = lineString.replace('{PHSXXmax}', str(PHSXXmax))
    
    if lineString.find('{PHSXXmin}'):
     lineString = lineString.replace('{PHSXXmin}', str(PHSXXmin))
    
    if lineString.find('{RHOXXERRmax}'):
     lineString = lineString.replace('{RHOXXERRmax}', str(RHOXXERRmax))
    
    if lineString.find('{RHOXXERRmin}'):
     lineString = lineString.replace('{RHOXXERRmin}', str(RHOXXERRmin))
    
    if lineString.find('{PHSXXERRmax}'):
     lineString = lineString.replace('{PHSXXERRmax}', str(PHSXXERRmax))
    
    if lineString.find('{PHSXXERRmin}'):
     lineString = lineString.replace('{PHSXXERRmin}', str(PHSXXERRmin))
    
    if lineString.find('{RHOXYmax}'):
     lineString = lineString.replace('{RHOXYmax}', str(RHOXYmax))
    
    if lineString.find('{RHOXYmin}'):
     lineString = lineString.replace('{RHOXYmin}', str(RHOXYmin))
    
    if lineString.find('{PHSXYmax}'):
     lineString = lineString.replace('{PHSXYmax}', str(PHSXYmax))
    
    if lineString.find('{PHSXYmin}'):
     lineString = lineString.replace('{PHSXYmin}', str(PHSXYmin))
    
    if lineString.find('{RHOXYERRmax}'):
     lineString = lineString.replace('{RHOXYERRmax}', str(RHOXYERRmax))
    
    if lineString.find('{RHOXYERRmin}'):
     lineString = lineString.replace('{RHOXYERRmin}', str(RHOXYERRmin))
    
    if lineString.find('{PHSXYERRmax}'):
     lineString = lineString.replace('{PHSXYERRmax}', str(PHSXYERRmax))
    
    if lineString.find('{PHSXYERRmin}'):
     lineString = lineString.replace('{PHSXYERRmin}', str(PHSXYERRmin))
    
    if lineString.find('{RHOYXmax}'):
     lineString = lineString.replace('{RHOYXmax}', str(RHOYXmax))
    
    if lineString.find('{RHOYXmin}'):
     lineString = lineString.replace('{RHOYXmin}', str(RHOYXmin))
    
    if lineString.find('{PHSYXmax}'):
     lineString = lineString.replace('{PHSYXmax}', str(PHSYXmax))
    
    if lineString.find('{PHSYXmin}'):
     lineString = lineString.replace('{PHSYXmin}', str(PHSYXmin))
    
    if lineString.find('{RHOYXERRmax}'):
     lineString = lineString.replace('{RHOYXERRmax}', str(RHOYXERRmax))
    
    if lineString.find('{RHOYXERRmin}'):
     lineString = lineString.replace('{RHOYXERRmin}', str(RHOYXERRmin))
    
    if lineString.find('{PHSYXERRmax}'):
     lineString = lineString.replace('{PHSYXERRmax}', str(PHSYXERRmax))
    
    if lineString.find('{PHSYXERRmin}'):
     lineString = lineString.replace('{PHSYXERRmin}', str(PHSYXERRmin))
    
    if lineString.find('{RHOYYmax}'):
     lineString = lineString.replace('{RHOYYmax}', str(RHOYYmax))
    
    if lineString.find('{RHOYYmin}'):
     lineString = lineString.replace('{RHOYYmin}', str(RHOYYmin))
    
    if lineString.find('{PHSYYmax}'):
     lineString = lineString.replace('{PHSYYmax}', str(PHSYYmax))
    
    if lineString.find('{PHSYYmin}'):
     lineString = lineString.replace('{PHSYYmin}', str(PHSYYmin))
    
    if lineString.find('{RHOYYERRmax}'):
     lineString = lineString.replace('{RHOYYERRmax}', str(RHOYYERRmax))
    
    if lineString.find('{RHOYYERRmin}'):
     lineString = lineString.replace('{RHOYYERRmin}', str(RHOYYERRmin))
    
    if lineString.find('{PHSYYERRmax}'):
     lineString = lineString.replace('{PHSYYERRmax}', str(PHSYYERRmax))
    
    if lineString.find('{PHSYYERRmin}'):
     lineString = lineString.replace('{PHSYYERRmin}', str(PHSYYERRmin))
    
    if lineString.find('{TIPMAGmax}'):
     lineString = lineString.replace('{TIPMAGmax}', str(TIPMAGmax))
    
    if lineString.find('{TIPMAGmin}'):
     lineString = lineString.replace('{TIPMAGmin}', str(TIPMAGmin))
    
    if lineString.find('{TIPPHSmax}'):
     lineString = lineString.replace('{TIPPHSmax}', str(TIPPHSmax))
    
    if lineString.find('{TIPPHSmin}'):
     lineString = lineString.replace('{TIPPHSmin}', str(TIPPHSmin))
    
    if lineString.find('{TipperAttributes}'):
     lineString = lineString.replace('{TipperAttributes}', str(TipperAttributes))
    
    if lineString.find('{ImpedanceAttributes}'):
     lineString = lineString.replace('{ImpedanceAttributes}', str(ImpedanceAttributes))
    
    if lineString.find('{TIPMAGERRmax}'):
     lineString = lineString.replace('{TIPMAGERRmax}', str(TIPMAGERRmax))
    
    if lineString.find('{TIPMAGERRmin}'):
     lineString = lineString.replace('{TIPMAGERRmin}', str(TIPMAGERRmin))
    
    if lineString.find('{TIPPHSERRmax}'):
     lineString = lineString.replace('{TIPPHSERRmax}', str(TIPPHSERRmax))
    
    if lineString.find('{TIPPHSERRmin}'):
     lineString = lineString.replace('{TIPPHSERRmin}', str(TIPPHSERRmin))
    
    if lineString.find('{ZSTRIKEmax}'):
     lineString = lineString.replace('{ZSTRIKEmax}', str(ZSTRIKEmax))
    
    if lineString.find('{ZSTRIKEmin}'):
     lineString = lineString.replace('{ZSTRIKEmin}', str(ZSTRIKEmin))
    
    if lineString.find('{ZSKEWmax}'):
     lineString = lineString.replace('{ZSKEWmax}', str(ZSKEWmax))
    
    if lineString.find('{ZSKEWmin}'):
     lineString = lineString.replace('{ZSKEWmin}', str(ZSKEWmin))
    
    if lineString.find('{TSTRIKEmax}'):
     lineString = lineString.replace('{TSTRIKEmax}', str(TSTRIKEmax))
    
    if lineString.find('{TSTRIKEmin}'):
     lineString = lineString.replace('{TSTRIKEmin}', str(TSTRIKEmin))
    
    if lineString.find('{COH1max}'):
     lineString = lineString.replace('{COH1max}', str(COH1max))
    
    if lineString.find('{COH1min}'):
     lineString = lineString.replace('{COH1min}', str(COH1min))
    
    if lineString.find('{COH2max}'):
     lineString = lineString.replace('{COH2max}', str(COH2max))
    
    if lineString.find('{COH2min}'):
     lineString = lineString.replace('{COH2min}', str(COH2min))
    
    if lineString.find('{COH3max}'):
     lineString = lineString.replace('{COH3max}', str(COH3max))
    
    if lineString.find('{COH3min}'):
     lineString = lineString.replace('{COH3min}', str(COH3min))
    
    if lineString.find('{COH4max}'):
     lineString = lineString.replace('{COH4max}', str(COH4max))
    
    if lineString.find('{COH4min}'):
     lineString = lineString.replace('{COH4min}', str(COH4min))
    
    if lineString.find('{EPREDCOH1max}'):
     lineString = lineString.replace('{EPREDCOH1max}', str(EPREDCOH1max))
    
    if lineString.find('{EPREDCOH1min}'):
     lineString = lineString.replace('{EPREDCOH1min}', str(EPREDCOH1min))
    
    if lineString.find('{EPREDCOH2max}'):
     lineString = lineString.replace('{EPREDCOH2max}', str(EPREDCOH2max))
    
    if lineString.find('{EPREDCOH2min}'):
     lineString = lineString.replace('{EPREDCOH2min}', str(EPREDCOH2min))
    
    if lineString.find('{SIGAMP1max}'):
     lineString = lineString.replace('{SIGAMP1max}', str(SIGAMP1max))
    
    if lineString.find('{SIGAMP1min}'):
     lineString = lineString.replace('{SIGAMP1min}', str(SIGAMP1min))
    
    if lineString.find('{SIGAMP2max}'):
     lineString = lineString.replace('{SIGAMP2max}', str(SIGAMP2max))
    
    if lineString.find('{SIGAMP2min}'):
     lineString = lineString.replace('{SIGAMP2min}', str(SIGAMP2min))
    
    if lineString.find('{SIGAMP3max}'):
     lineString = lineString.replace('{SIGAMP3max}', str(SIGAMP3max))
    
    if lineString.find('{SIGAMP3min}'):
     lineString = lineString.replace('{SIGAMP3min}', str(SIGAMP3min))
    
    if lineString.find('{SIGAMP4max}'):
     lineString = lineString.replace('{SIGAMP4max}', str(SIGAMP4max))
    
    if lineString.find('{SIGAMP4min}'):
     lineString = lineString.replace('{SIGAMP4min}', str(SIGAMP4min))
    
    if lineString.find('{SIGAMP5max}'):
     lineString = lineString.replace('{SIGAMP5max}', str(SIGAMP5max))
    
    if lineString.find('{SIGAMP5min}'):
     lineString = lineString.replace('{SIGAMP5min}', str(SIGAMP5min))
    
    if lineString.find('{SIGNOISE1max}'):
     lineString = lineString.replace('{SIGNOISE1max}', str(SIGNOISE1max))
    
    if lineString.find('{SIGNOISE1min}'):
     lineString = lineString.replace('{SIGNOISE1min}', str(SIGNOISE1min))
   
    if lineString.find('{SIGNOISE2max}'):
     lineString = lineString.replace('{SIGNOISE2max}', str(SIGNOISE2max))
    
    if lineString.find('{SIGNOISE2min}'):
     lineString = lineString.replace('{SIGNOISE2min}', str(SIGNOISE2min))
   
    if lineString.find('{SIGNOISE3max}'):
     lineString = lineString.replace('{SIGNOISE3max}', str(SIGNOISE3max))
    
    if lineString.find('{SIGNOISE3min}'):
     lineString = lineString.replace('{SIGNOISE3min}', str(SIGNOISE3min))
   
    if lineString.find('{SIGNOISE4max}'):
     lineString = lineString.replace('{SIGNOISE4max}', str(SIGNOISE4max))
    
    if lineString.find('{SIGNOISE4min}'):
     lineString = lineString.replace('{SIGNOISE4min}', str(SIGNOISE4min))
   
    if lineString.find('{SIGNOISE5max}'):
     lineString = lineString.replace('{SIGNOISE5max}', str(SIGNOISE5max))
    
    if lineString.find('{SIGNOISE5min}'):
     lineString = lineString.replace('{SIGNOISE5min}', str(SIGNOISE5min))
   
    # {county}

    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

C:\Temp\MT1-DR\Animas-AMT\amtsf06\USA-Colorado-Silverton_Caldera-2019-AMTSF06.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/

In [98]:
# Show the resulting child xml metadata file example 
#for i in range(len(newMetaDataContent)):
print (newMetaDataContent)

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '\t\t\t\t<origin>Reitman, J.</origin>\n', '\t\t\t\t<origin>Rodriguez, B.D.</origin>\n', '\t\t\t\t<origin>Brown, P.J.</origin>\t\t\t\t\t\t\t\t\n', '\t\t\t\t<pubdate>2020</pubdate>\n', '\t\t\t\t<title>{title}</title>\n', '\t\t\t\t<edition>1</edition>\n', '\t\t\t\t<geoform>ASCII and Binary Digital Data</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators: Reitman, J., http://orcid.org/0000-0003-3551-9884; Rodriguez, B.D., http://orcid.org/0000-0002-2263-611X; Brown, P.J., http://orcid.org/0000-0002-2415-7462</othercit>\n', '\t\t\t\t<onlink>https://doi.org/10.5066/P9F68AQP</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', '\t\t\t<abstract>This dataset includes audiomagnetotellu